# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# check current work directory
print(os.getcwd())

# Get current folder and subfolder for event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
#total number of rows in ull data
print("Total Number of rows in full data: " + str(len(full_data_rows_list)))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))



Total Number of rows in full data: 8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
#Creating Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set up keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# create artist_songs table
query = "CREATE TABLE IF NOT EXISTS artist_songs"
query = query + "(Artist varchar, \
                  Song varchar, \
                  Length float, \
                  SessionId int, \
                  IteminSession int, \
                  PRIMARY KEY (SessionId, IteminSession))"

# execute create table query
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# insert records into artist_songs table from csv
file = 'event_datafile_new.csv'

# open csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
# assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_songs(Artist, \
                                          Song, \
                                          Length, \
                                          SessionId, \
                                          IteminSession)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        
# execute Insert Query for artist_songs table
        try:
            session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3]), ))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# create SELECT query for artist_songs table to find artist, song and lengh of song where sessionid=338 and iteminsession=4
query = "SELECT Artist, \
                Song, \
                Length \
         from artist_songs \
         where sessionid = 338 and \
               iteminsession = 4  "

# execute Select Query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#Print artist_songs output using prettytable
artist_songs_pt = PrettyTable()
artist_songs_pt.field_names = ["Artist","Song","Length"]
for row in rows:
    artist_songs_pt.add_row((row.artist, row.song, row.length))
print("Artists and Songs where sessionid = 338 and iteminsession = 4")
print(artist_songs_pt)

Artists and Songs where sessionid = 338 and iteminsession = 4
+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# create artist_songs_users table
query = "CREATE TABLE IF NOT EXISTS artist_songs_users"
query = query + "(Artist varchar, \
                  Song varchar, \
                  FirstName varchar, \
                  LastName varchar, \
                  UserId int, \
                  SessionId int, \
                  ItemInSession int, \
                  PRIMARY KEY ((UserId, SessionId), ItemInSession))"

# execute create query for artist_songs_users
try:
    session.execute(query)
except Exception as e:
    print(e)

# insert records into artist_songs_users table from csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
# assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_songs_users(Artist, \
                                                Song, \
                                                FirstName, \
                                                LastName, \
                                                UserId, \
                                                SessionId, \
                                                ItemInSession)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
# execute insert query for artist_songs_users
        try:
            session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
        except Exception as e:
            print(e)
            

# create SELECT query for artist_songs_users table to find artist, song and users 
# where userid=10 and sessioni=182 and orderd by iteminsession=4
query = "SELECT Artist, \
                Song, \
                FirstName, \
                LastName \
         from artist_songs_users \
         where UserId = 10 and SessionId = 182 "

#execute select query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#Print artist_songs_users output using prettytable
artist_songs_users_pt = PrettyTable()
artist_songs_users_pt.field_names = ["Artist","Song","FirstName","LastName"]
for row in rows:
    artist_songs_users_pt.add_row((row.artist, row.song, row.firstname, row.lastname))
print("Artists, Songs and Name for UserId = 10 and SessionId = 182")
print(artist_songs_users_pt)

Artists, Songs and Name for UserId = 10 and SessionId = 182
+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | LastName |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


In [12]:
# create users_name table
query = "CREATE TABLE IF NOT EXISTS users_name"
query = query + "(FirstName varchar, \
                  LastName varchar, \
                  Song varchar, \
                  UserId int, \
                  PRIMARY KEY (Song, UserId))"

# execute create query
try:
    session.execute(query)
except Exception as e:
    print(e)   

# insert records into users_name table from csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
# assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_name(FirstName, \
                                        LastName, \
                                        Song, \
                                        UserId)"
        query = query + " VALUES(%s, %s, %s, %s)"
        
        ## execute insert query for users_name
        try:
            session.execute(query, (line[9], line[1], line[4], int(line[10])))
        except Exception as e:
            print(e)
            
# create SELECT query for users_name table to find users who listend song 'All Hands Against His Own' 
query = "SELECT FirstName, \
                LastName \
         from users_name \
         where song = 'All Hands Against His Own' "

# execute select query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#Print users_name output using prettytable
users_name_pt = PrettyTable()
users_name_pt.field_names = ["FirstName","LastName"]
for row in rows:
    users_name_pt.add_row((row.firstname, row.lastname))
print("Users who listend to song 'All Hands Against His Own'")
print(users_name_pt)

Users who listend to song 'All Hands Against His Own'
+-----------+----------+
| FirstName | LastName |
+-----------+----------+
+-----------+----------+


### Drop the tables before closing out the sessions

In [13]:
# drop the table before closing out the sessions
print("Droping... artist_songs table")
query = "drop table artist_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print("Droping... artist_songs_users table")
query = "drop table artist_songs_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print("Droping... users_name table")
query = "drop table users_name"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Droping... artist_songs table
Droping... artist_songs_users table
Droping... users_name table


### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()